<a href="https://colab.research.google.com/github/Existanze54/sirius-neural-networks-2024/blob/main/Lections/08L_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекуррентные нейронные сети (RNN)

Рекуррентные нейронные сети $-$ подмножество нейронных сетей, разработанное для работы с неструктурированными данными, представляющими собой некие последовательности объектов, не имеющие связной структуры и признаковых описаний. К примерам таких аднных можно отнести звуковые ряды, видео, временные ряды и конечно же текст. За счет своего устройства нейронные сети способны выделять закономерности, содержащиеся в таких последовательностях, запоминать, и использовать эту информацию при дальнейшем анализе последовательности.

Название "рекуррентные" проистекает из наличия в сети виртуального набора последовательных однотипных ячеек, через которые и пропускается информация. В реальности в нашей модели в рекуррентном слое такая ячейка только одна, однако при пропускании через себя последовательности она изменяет свое состояние и "запоминает" то, что видела ранее.

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" alt="Drawing" width= "700px;"/>
<a href="https://colah.github.io/posts/2015-08-Understanding-LSTMs/">Source</a>



Варианты архитектуры рекуррентный нейронных сетей не ограничиваются схемой "один вход - один выход", что используется, например, для классификации изображений. One-to-One -- схема, с которой мы работали ранее: принимаем на вход 1 объект, обрабатываем его полностью, выдаем одно предсказание (например, метку класса).<br>

<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/diags.jpeg" alt="Drawing" width= "800px;"/>

Рекуррентные нейронные сети позволяют решать и другие задачи.
Чуть более сложным вариантом является схема "one_to-many", подразумевающая генерацию последовательности объектов в ответ на один поданный на вход. Например, такую схему используют для генерации текстового описания картинки.<br>
Существуют варианты и обратные предыдущему, например, для создания изображения по его текстовому описанию.<br>
Наконец, возможны архитектуры вида "many-to-many", генерирующие последовательность в ответ на последовательность. Такие могут быть использованы для перевода текстов или генерации покадрового описания к видео. Такие архитектуры могут выдавать результат после полного поглощения исходной последовательности, для лучшего выделения контекста, или сразу во время чтения, например, когда нужен синхронный перевод текста.

([Источник](http://karpathy.github.io/2015/05/21/rnn-effectiveness/))

## Базовая ячейка RNN  (RNNCell)

Для начала посмотрим на то, что происходит внутри одной рекуррентной ячейки. <br>
В отличие от нейрона "обычной" сети, рекуррентная ячейка принимает два входа: элемент последовательности $X_t$ и предыдущее скрытое состояние $h_{t-1}$. Текущее скрытое состояние вычисляется при помощи взвешенного сложения входов, после чего может быть вычисле выход, а текущее скрытое состояние передается далее и может быть использовано для вычисления $h_{t+1}$.


$$h_t = tanh(W_{hh}h_{t-1} + W_{xh}x_t)$$


$$y_t = W_{hy}h_t$$

<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/rnncell.png" alt="Drawing" width= "500px;"/>


Важно понимать, что веса, использующиеся для получения скрытого состояния, во время предсказания остаются неизменными для всех элементов последовательности (и именно они ответственны за обучение RNN ячейки).

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" alt="Drawing" width= "700px;"/>

## Блок RNN

В Pytorch реализованы готовые классы как для базовой рекуррентной ячейки RNNCell, так и для настоящего рекуррентного слоя под названием [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html). Этот класс освобождает нас от необходимости писать циклы и позволяет применить внутреннюю RNN ячейку ко всех элементам поданной на вход последовательности. При создании ячейки изначальный хидден стейт по дефолту заполняется нулями, однако можно инициализировать его любым тензором нужной формы.

Важно обратить внимание на то, что по умолчанию батчевая размерность подразумевается второй по счету. Для управления этим поведением предусмотрен булевый аргумент **batch_first**.



In [ ]:
from torch import nn
rnn = torch.nn.RNN(input_size = 3, hidden_size = 2) # batch_first = False
dummy_batched_seq = torch.randn((2,1,3)) # seq_len, batch , input_size
out, h = rnn(dummy_batched_seq)

print(f"Dummy: \n {dummy_batched_seq.shape}\n {dummy_batched_seq}")
print(f"Out: \n {out.shape}\n {out}") # hidden state for each element of sequence
print(f"h: \n {h.shape}\n {h}") # hidden state for last element of sequence

Dummy: 
 torch.Size([2, 1, 3])
 tensor([[[-1.8667,  1.2638,  0.0826]],

        [[ 0.1110, -0.5911, -0.8078]]])
Out: 
 torch.Size([2, 1, 2])
 tensor([[[-0.4575, -0.0697]],

        [[-0.4038,  0.2472]]], grad_fn=<StackBackward>)
h: 
 torch.Size([1, 1, 2])
 tensor([[[-0.4038,  0.2472]]], grad_fn=<StackBackward>)


Попробуем написать простую рекуррентную нейросетку!<br>
Добавим выходной полносвязный слой, принимающий на вход последнее скрытое состояние. <br>
Обратите внимание на то, что из-за особенностей торча у выхода будет на одну размерность больше. Такая "лишняя" размерность может быть весьма надоедливой, и лучше её сразу убрать за ненадобностью.

In [ ]:
import torch
# Let's add output weights

class RNN_for_many_to_one(torch.nn.Module):
    def __init__(self, input_size, hidden, output_size):
        super().__init__()
        self.rnn = torch.nn.RNN(input_size, hidden, batch_first=True) # RNN слой
        self.fc1 = torch.nn.Linear(hidden, output_size)                 # Полносвязный слой

    def forward(self, x):
        x, hidden = self.rnn(x)
        print("All hidden states:\t", x.shape) # h for each element
        print("Last hidden state:\t", hidden.shape)
        # we need only last output
        # return self.fc1(x[-1])
        return self.fc1(hidden)

model2 = RNN_for_many_to_one(28, 128, 10) # input_size, hidden_dim, classes
dummy_input = torch.randn((8, 28, 28)) #  batch, seq_len, element_size
res = model2(dummy_input)

print("Annoying extra dimension:", res.shape)
print("Real output:\t\t", res[0].shape)

All hidden states:	 torch.Size([8, 28, 128])
Last hidden state:	 torch.Size([1, 8, 128])
Annoying extra dimension: torch.Size([1, 8, 10])
Real output:		 torch.Size([8, 10])


## Bidirectional RNNs

Последовательность можно пропустить через сеть два раза, в прямом и обратном направлении.

Идея тут в том, что идя по предложению, мы не знаем что у него будет в конце. А иногда это нужно по постановке задачи (например, перевод). Таким образом, на каждом шаге будет собираться информация с начала и конца предложения ( и понемногу затухать). Иногда это работает лучше. Отвечает за такую архитектуру параметр **bidirectional**.

<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/bidirectional.png" alt="Drawing" width= "700px;"/>




In [ ]:
import torch

dummy_input = torch.randn((2,1,3)) #   seq_len, batch, input_size
rnn = torch.nn.RNN(3, 2, bidirectional=True, num_layers=1)

for t, p in rnn.named_parameters():
  print(t, p.shape)

out, h = rnn(dummy_input)
print("")
print("Out", out) # Concatenated Hidden states from both layers
print("h", h) # Hidden states last element from  both : 2*num_layers*hidden_state

weight_ih_l0 torch.Size([2, 3])
weight_hh_l0 torch.Size([2, 2])
bias_ih_l0 torch.Size([2])
bias_hh_l0 torch.Size([2])
weight_ih_l0_reverse torch.Size([2, 3])
weight_hh_l0_reverse torch.Size([2, 2])
bias_ih_l0_reverse torch.Size([2])
bias_hh_l0_reverse torch.Size([2])

Out tensor([[[-0.8525, -0.0962,  0.4507, -0.1198]],

        [[ 0.3427,  0.1302,  0.4982, -0.2478]]], grad_fn=<CatBackward>)
h tensor([[[ 0.3427,  0.1302]],

        [[ 0.4507, -0.1198]]], grad_fn=<StackBackward>)


## LSTM

В конце 90-х важным прорывом в нейросетевой науке было изобретение ячеек *long short-term memory*. Важным отличием от обычной RNN ячейки является наличие новой "трассы", в которой аккумулируется (и со временем "забывается") результат объединения всех предыдущих скрытых состояний. Таким образом, ячейка помимо собственно скрытого состояния обладает "**памятью**".

<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/lstm.png" alt="Drawing" width= "700px;"/>

Рассмотрим элементы LSTM-ячейки по отдельности и разберемся по-подробнее с тем, что же здесь происходит. Уже упомянутая трасса памяти обозначена как горизонтальная линия, бегущая сквозным конвейером через ячейки:   

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-C-line.png" alt="Drawing" width= "700px;"/>

Помимо этой трассы, в LSTM-ячейке реализованы дополнительные операции, отвечающие за добавление и удаление информации из нее. Количество информации, пропускаемое через эти дополнительные каналы, регулируется так называемыми "шлюзами" при помощи сигмоиды, которая и принимает решение,  какие значения оставить, а какие отбросить в каждом случае. Таких шлюзов реализовано 3:

1. **Forget gate**, или Шлюз Забывания отвечает за удаление информации из трассы состояния ячейки. Информация, которая больше не требуется LSTM для понимания вещей, или менее важная информация удаляется путем умножения на фильтр - матрицу весов. Это необходимо для оптимизации производительности сети LSTM. На вход принимает предыдущее скрытое состояние $h_{t-1}$ и текущий элемент последовательности $x_t$. Заданные входные данные умножаются на матрицы весов и добавляется смещение. После применения сигмоиды полученный фильтр применяется к трассе памяти.

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png" alt="Drawing" width= "700px;"/>

2. **Input gate** или входной шлюз. Похож на предыдущий шаг, но действует в обратном направлении: ячейка принимает решение о добавлении новой информации в трассу памяти. Это делается в два этапа: сначала при помощи сигмоиды принимается решение, какие именно значения мы обновим (получаем тензор индексов $i_t$), а при помощи тангенса мы получаем собственно новые значения $C_t$, пригодные к добавлению в трассу. Обратите внимание, что для этих шагов мы обучаем разные матрицы весов. Совместив два тензора на этом шаге мы готовы добавить новую инормацию в трассу.  

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png" alt="Drawing" width= "700px;"/>



3. Выходной шлюз, или **Output gate** принимает решение о том, какой же хидден стейт ячейка вернет для текущего элемента последовательности $x_t$. Для этого мы отфильтруем его при помощи сигмоиды и после этого умножим на текущее состояние памяти ячейки, пропущенное черз тангенс (чтобы значения не выходили за пределы -1..1).

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png" alt="Drawing" width= "700px;"/>

Следующее состояние ячейки примет на вход память, хидден стейт и следующий элемент последовательности.




## GRU

GRU, или Gated Reccurent Unit, являются еще одной версией RNN с подобием памяти.

У этой версии не три в хода, а два: нет "трассы" внутреннего состояния, отличного от скрытого. Вместо этого, два шлюза отвечают за обновление и забывание информации в скрытом состоянии, которое и передается дальше без дополнительного наложения нелинейности.

<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/gru.png" alt="Drawing" width= "900px;"/>

## Как это применяется в биологии?

Долгое время RNN были в биологии весьма сильным методом как раз из-за своей способности работать с биологическими последовательностями. Впрочем, даже тогда с ними достаточно сильно конкурировали сверточные сети.

На сегодняшний день все еще можно использовать RNN для каких-то не сильно сложных задач, но нет никакой гарантии, что сверточная модель вас не обойдёт.


Всё же, "в чистом виде" модели, в которых RNN использовался бы в качестве входного блока, обычно достаточно неглубокие (1 слой RNN + 1 слой fc классификатор) и используются для каких-то сравнительно просто решаемых задач, либо в задачах text-майнинга.

DOI: 10.1007/s00251-013-0720-y

Предсказание связывания пептидов с MHC II


<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/example3.png" alt="Drawing" width= "600px;"/>

Гораздо чаще встречаются архитектуры, в которых входные слои сверточные, позволяющие получить из one-hot закодированных последовательностей/выравниваний более "плотные" эмбеддинги, которые уже подаются на вход рекуррентным блокам, а их выход -- уже полносвязным слоям.


Предсказание клеточной локализации белка на основе последовательности.

doi: 10.1093/bioinformatics/btx531

<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/example4.png" alt="Drawing" width= "600px;"/>

Достаточно красивая статья. Пожалуй, одна из последних статей, когда "чистым" рекуррентным сетям ничто не угрожало на их поприще. Уже через год, в 2017, будут представлены первые трансформеры...


<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/dcrnn.png" alt="Drawing" width= "900px;"/>


https://arxiv.org/abs/1604.07176 <br>
https://github.com/icemansina/IJCAI2016


В статье представлена сеть, предсказывающая вторичную структуру белка по последовательности для соревнования CASP (и на момент выхода превзошедшая всех конкурентов на то время). Принимала на вход эмбеддинг последовательностей и фичи из профиля пси-бласта, после конкатенации вход пропускается через свертки и отдается на растеразание двунаправленным GRUшкам (обоснование: поиск идет в два направления, потому что участие каждого остатка в структуре задается его двумя соседями с обеих сторон), на выходе полносвязные слои предсказывают вероятность каждого остатка принадлежать тому или иному элементу вторичной структуры и -- для валидации -- доступность для растворителя.

Интересно, что для реализации модели авторы выбрали умиравший уже тогда фреймворк Theano.

## Чем всё закончилось?
На следующей лекции вы поближе познакомитесь с трансформерами --  моделями, которые во многих задачах стали выступать лучше RNN как по эффективности, так и по времени работы.



<img src="https://kodomo.fbb.msu.ru/FBB/year_20/ml/rnn/trans1.png" alt="Drawing" width= "500px;"/>